# Regression test suite for the SAMBAH:

This notebook provides condensed examples of using Harmony to make requests against the SAMBAH service developed to process L2 TEMPO data. The features of SAMBAH include:

* Variable subsetting, including required variables.
* Temporal subsetting.
* Bounding box spatial subsetting.

## Prerequisites

The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill-sambah`

A `.netrc` file must also be located in the `test` directory of this repository.

## Import requirements:

In [ ]:
from datetime import datetime
from os.path import exists

from harmony import BBox, Client, Collection, Dimension, Environment, Request

from utilities import (
    compare_results_to_reference_file,
    remove_results_files,
    submit_and_download,
)

## Set default parameters:

`papermill` requires default values for parameters used on the workflow. In this case, `harmony_host_url`.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

### Identify Harmony environment (for easier reference):

In [ ]:
host_environment = {
    'http://localhost:3000': Environment.LOCAL,
    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
    'https://harmony.earthdata.nasa.gov': Environment.PROD,
}

data_environment = 'PROD' if harmony_host_url == '' else 'UAT'
req_infos = {
    'UAT': {
        # TEMPO NO2 tropospheric, stratospheric, and total columns V03
        'collection': Collection(id='C1262899916-LARC_CLOUD'),
        'temporal': {
            'start': datetime(2024, 3, 28, 15, 34, 0),
            'stop': datetime(2024, 3, 28, 16, 0, 0),
        },
        'spatial': BBox(-170, 33, -10, 38),
        'granule_id': [
            'G1263137623-LARC_CLOUD',
            'G1263137394-LARC_CLOUD',
            'G1263137387-LARC_CLOUD',
            'G1263137388-LARC_CLOUD',
            'G1263137378-LARC_CLOUD',
        ],
        # chosen variables include one variable from each group
        # support/scattering_weights is 3D variable
        'variables': [
            'product/vertical_column_stratosphere',
            'qa_statistics/fit_rms_residual',
            'support_data/scattering_weights',
        ],
    },
}

req_info = req_infos.get(data_environment)
harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

# Begin regression tests:

## SAMBAH

SAMBAH is currently deployed to Sandbox, SIT, UAT and production.
Requests will be made against the TEMPO NO2 L2 V03 collection.

### SAMBAH: temporal, variable and bounding box subset request

This is a request that exercises the full range of SAMBAH options: temporal, bounding box and variable subsetting.

Requested variable names:

* `2024-3-28 15:34:00 < time < 2024-3-28 16:00:00`
* `product/vertical_column_total`
* `BBox(-170,30,-10,40)`

In [ ]:
temp_var_bbox_request = Request(
    collection=req_info['collection'],
    extend=['mirror_step'],
    granule_id=req_info['granule_id'],
    temporal=req_info['temporal'],
    variables=req_info['variables'],
    spatial=req_info['spatial'],
)


temp_var_bbox_filename = 'temp_var_bbox.nc4'
submit_and_download(harmony_client, temp_var_bbox_request, temp_var_bbox_filename)
assert exists(
    temp_var_bbox_filename
), 'Unsuccessful SAMBAH temporal, variable, bounding box request.'


compare_results_to_reference_file(temp_var_bbox_filename)
compare_results_to_reference_file(temp_var_bbox_filename, 'product')
# compare_results_to_reference_file(temp_var_bbox_filename, 'geolocation')
compare_results_to_reference_file(temp_var_bbox_filename, 'qa_statistics')
compare_results_to_reference_file(temp_var_bbox_filename, 'support_data')


print("SAMBAH temporal, variable, bounding box request: success")

### SAMBAH: variable subset request

In [ ]:
var_only_request = Request(
    collection=req_info['collection'],
    extend='mirror_step',
    granule_id=req_info['granule_id'],
    variables=req_info['variables'],
)

var_only_filename = 'var_only.nc4'
submit_and_download(harmony_client, var_only_request, var_only_filename)
assert exists(var_only_filename), 'Unsuccessful SAMBAH variable request.'

compare_results_to_reference_file(var_only_filename)
print("SAMBAH variable request: success")

### SAMBAH: spatial request

In [ ]:
xr.open_dataset("var_only.nc4")

In [ ]:
spatial_only_request = Request(
    collection=req_info['collection'],
    extend='mirror_step',
    granule_id=req_info['granule_id'],
    spatial=req_info['spatial'],
)

spatial_only_filename = 'spatial_only.nc4'
submit_and_download(harmony_client, spatial_only_request, spatial_only_filename)
assert exists(spatial_only_filename), 'Unsuccessful SAMBAH temporal request.'

compare_results_to_reference_file(spatial_only_filename)
print("SAMBAH spatial request: success")

### SAMBAH: no subsetting required


In [ ]:
all_data_request = Request(
    collection=req_info['collection'],
    extend='mirror_step',
    granule_id=req_info['granule_id'],
)

all_data_filename = 'all_data.nc4'
submit_and_download(harmony_client, all_data_request, all_data_filename)
assert exists(
    all_data_filename
), 'Unsuccessful SAMBAH temporal, variable, bounding box request.'

compare_results_to_reference_file(all_data_filename)
print("SAMBAH temporal, variable, bounding box request: success")

# Clean up test outputs:

In [ ]:
remove_results_files()